***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-6d1a  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  8 12:49 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'nettaya-315443382' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

In [8]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [9]:
from inverted_index_gcp import *

In [10]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("id", "title").rdd
inverted_title = InvertedIndex()

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


def word_count(id, text):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,(id,tf_dict[t])))
  return tuples

def count_doc_len(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (doc_id,len(tokens))

def tf_for_term_id(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,tf_dict[t]))
  return tuples

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  token_df = postings.map(lambda x: (x[0], len(x[1])))
  return token_df



In [12]:
word_counts_title = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
len_docs_title = doc_text_pairs.map(lambda x: count_doc_len(x[0], x[1]))
len_docs_title = len_docs_title.collectAsMap()
inverted_title.DL = len_docs_title

In [13]:
first_20_items = dict(list(inverted_title.DL.items())[:20])
print(first_20_items)


{4045403: 4, 4045413: 2, 4045419: 2, 4045426: 2, 4045432: 1, 4045456: 3, 4045466: 2, 4045471: 2, 4045479: 2, 4045516: 2, 4045519: 2, 4045523: 1, 4045525: 5, 4045532: 1, 4045544: 2, 4045546: 1, 4045554: 4, 4045575: 2, 4045577: 2, 4045581: 4}


In [14]:
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [15]:
total_terms_title = postings_title.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_title.term_total = total_terms_title.collectAsMap()

In [16]:
first_20_items = dict(list(inverted_title.term_total.items())[:20])
print(first_20_items)

{'glass': 1247, 'crawfordsville': 20, 'los': 4402, '1983': 3192, 'field': 6356, 'east': 10466, 'denise': 439, 'engine': 1570, 'bar': 1574, 'mair': 90, 'tracy': 638, 'silver': 2090, 'carr': 771, 'composer': 1362, 'chrysalis': 29, 'nawaz': 123, 'mind': 1185, 'abdel': 399, 'sherman': 723, 'raaba': 1}


In [17]:
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()


In [18]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,base_dir):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  bucket_name = "nettaya-315443382"

 # Assume postings is an RDD or a similar collection where each item is a (word, posting list) pair
 # First, transform postings into (bucket_id, (word, posting list)) for grouping using your `token2bucket_id` logic
  postings_by_bucket = postings.map(lambda x: (token2bucket_id(x[0]), x))

  # Group postings by bucket_id and prepare them for writing
  postings_grouped = postings_by_bucket.groupByKey().mapValues(list)

  # For each bucket, write postings to disk and collect location info
  postings_locations = postings_grouped.map(lambda x: InvertedIndex.write_a_posting_list(x, base_dir, bucket_name))

  return postings_locations

In [19]:
postings_locs_title = partition_postings_and_write(postings_title,'title').collect()

In [20]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title'):
    if not blob.name.endswith(".pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_title[k].extend(v)


In [21]:
first_20_items = dict(list(super_posting_locs_title.items())[:20])
print(first_20_items)

{'montgomery': [('title/0_000.bin', 0)], 'argentino': [('title/0_000.bin', 7092)], 'andronikos': [('title/0_000.bin', 8070)], 'ricci': [('title/0_000.bin', 8400)], 'ardenwald-johnson': [('title/0_000.bin', 9324)], 'shinwar': [('title/0_000.bin', 9330)], 'gunzgen': [('title/0_000.bin', 9342)], 'chocolat': [('title/0_000.bin', 9348)], 'woolly': [('title/0_000.bin', 9510)], 'abertrinant': [('title/0_000.bin', 10026)], 'butega': [('title/0_000.bin', 10032)], 'piri': [('title/0_000.bin', 10038)], 'baczyńscy': [('title/0_000.bin', 10230)], 'dąbrowa-wilki': [('title/0_000.bin', 10236)], 'kwamina': [('title/0_000.bin', 10242)], 'selemani': [('title/0_000.bin', 10302)], 'porsche-schinkeová': [('title/0_000.bin', 10350)], 'olit': [('title/0_000.bin', 10356)], 'quétineau': [('title/0_000.bin', 10362)], 'juruti': [('title/0_000.bin', 10368)]}


In [22]:
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_title_dict
# write the global stats out
inverted_title.write_index('.', 'index_title')
!ls -l index_title.pkl
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp index_title.pkl gs://nettaya-315443382/postings_gcp_title/index_title.pkl

-rw-r--r-- 1 root root 139370875 Mar  8 13:29 index_title.pkl
Copying file://index_title.pkl [Content-Type=application/octet-stream]...
\ [1 files][132.9 MiB/132.9 MiB]                                                
Operation completed over 1 objects/132.9 MiB.                                    


In [23]:
first_20_items = dict(list(inverted_title.posting_locs.items())[:20])
print(first_20_items)

{'montgomery': [('title/0_000.bin', 0)], 'argentino': [('title/0_000.bin', 7092)], 'andronikos': [('title/0_000.bin', 8070)], 'ricci': [('title/0_000.bin', 8400)], 'ardenwald-johnson': [('title/0_000.bin', 9324)], 'shinwar': [('title/0_000.bin', 9330)], 'gunzgen': [('title/0_000.bin', 9342)], 'chocolat': [('title/0_000.bin', 9348)], 'woolly': [('title/0_000.bin', 9510)], 'abertrinant': [('title/0_000.bin', 10026)], 'butega': [('title/0_000.bin', 10032)], 'piri': [('title/0_000.bin', 10038)], 'baczyńscy': [('title/0_000.bin', 10230)], 'dąbrowa-wilki': [('title/0_000.bin', 10236)], 'kwamina': [('title/0_000.bin', 10242)], 'selemani': [('title/0_000.bin', 10302)], 'porsche-schinkeová': [('title/0_000.bin', 10350)], 'olit': [('title/0_000.bin', 10356)], 'quétineau': [('title/0_000.bin', 10362)], 'juruti': [('title/0_000.bin', 10368)]}


In [24]:
first_20_items = dict(list(inverted_title.df.items())[:20])
print(first_20_items)

{'tianjin': 205, 'lambs': 29, 'streptanthus': 32, 'bread': 341, 'henan': 161, 'glass': 1244, 'akina': 33, 'los': 4388, 'demon': 493, 'field': 6350, 'padvaxtag': 1, 'herrera': 489, 'interaction': 156, 'z-403': 1, 'trunking': 8, 'east': 10448, 'cantigny': 5, 'gunerius': 7, 'stockholm': 529, 'jamaica': 841}
